In [ ]:
import numpy as np
import matplotlib as mtp
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import rasterio
from rasterstats import zonal_stats
import os
import glob
from rasterio.plot import show
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import train_test_split, GridSearchCV
from mlxtend.feature_selection import SequentialFeatureSelector
from sklearn.metrics import root_mean_squared_error, mean_absolute_error
from sklearn.inspection import permutation_importance
import optuna
import joblib

###### The ff lines of code imports crop field boundaries and an already preprocessed time series of optical VIs and SAR data pairs.
###### For each crop field boundary, the satellite data response are averaged to minimize SAR speckle noise.  

In [ ]:
# Set the base directory
os.chdir('C:/raster_vector')

# Read file names into lists
img_files = glob.glob('*tif')
shapefiles = glob.glob('*shp')

# Function to read data: opens .tif files and reads .shp files
def read_data(file):
    if file.endswith('tif'):
        return rasterio.open(file)
    else:
        return gpd.read_file(file)

# Store the arrays of optical/SAR images and field boundaries in lists
opt_SAR = [read_data(img) for img in img_files]
boundary = [read_data(vector) for vector in shapefiles]

# Read rasterio objects into numpy arrays
img_stack = [img.read() for img in opt_SAR]

# Transform pixel coordinates to x, y map coordinates for each image stack
affines = [img.transform for img in opt_SAR]

# Calculate the mean RS feature value for each field boundary, for all acquisition dates
mean = []
for stack, poly, affine in zip(img_stack, boundary, affines):
    for band in stack:
        mean_df = pd.DataFrame(zonal_stats(poly, band, affine=affine, stats='mean'))
        mean.append(mean_df)

# Rename all the columns in the 'mean' list with their respective spectral band names and their acquisition date
names = ['VV', 'VH', 'RVI', 'Ratio', 'NDVI', 'EVI']
col_names = [filename[:-4] + '_' + name for filename in shapefiles for name in names]

# Apply the new column names to the DataFrames
for df, col_name in zip(mean, col_names):
    df.rename(columns={'mean': col_name}, inplace=True)

In [ ]:
# Split the 'mean' list into chunks of 6 items and concatenate them into dataframes
chunks = [mean[i:i+6] for i in range(0, len(mean), 6)]
dfs_mean = [pd.concat(chunk, axis=1) for chunk in chunks]

# Filter necessary columns in the field boundary shapefiles
field_bounds = [poly.loc[:, ['OBJECTID', 'CDL' + shape[2:6], 'geometry']] for poly, shape in zip(boundary, shapefiles)]

# Append map coordinates and crop ID to each dataframe
gdfs = [pd.concat([field, df], axis=1) for df, field in zip(dfs_mean, field_bounds)]

# Add date and region columns for each dataframe using optical data acquisition date
for df in gdfs:
    date_str = df.columns[3][2:12]
    df['Date'] = pd.to_datetime(date_str[:4] + '-' + date_str[5:7] + '-' + date_str[8:])
    df['Region'] = df.columns[3][0]

# Rename columns for row-wise concatenation
new_names = ['OBJECTID', 'CDL', 'geometry', 'VV', 'VH', 'RVI', 'Ratio', 'NDVI', 'EVI', 'Date', 'Region']
for df in gdfs:
    df.columns = new_names

# Reproject all geodataframes to EPSG 4326 and concatenate them row-wise
gdfs_proj = [df.to_crs('epsg:4326') for df in gdfs]
gdf = pd.concat(gdfs_proj).reset_index(drop=True)

# Drop rows with NaN or VV==0.0
gdf = gdf.dropna(subset=['VV']).loc[gdf['VV'] != 0.0].reset_index(drop=True)

#NB: the values of the two columns below were split between string and integer data types, so I had to convert them into one before saving the data
# Convert OBJECTID and CDL columns to string datatype
gdf[['OBJECTID', 'CDL']] = gdf[['OBJECTID', 'CDL']].astype('str')

# Save the final dataframe to a geoparquet file
gdf.to_parquet('E:/New_thesis/HLS_S1_RUN/HLS_S1.geoparquet')


In [12]:
# Load geoparquet files 
g_data = gpd.read_parquet('C:/Users/henry/Documents/New_thesis/output/HLS_S1.geoparquet')

no_DC = gpd.read_parquet('E:/New_thesis/HLS_S1_RUN/no_double_cropping.geoparquet') #Occitanie data with single and double-cropping
no_DC.rename(columns={'ID_PARCEL': 'OBJECTID'}, inplace=True)

# Update CDL and Region columns
g_data['CDL'] = g_data['CDL'].replace({'5': 'Soyabean', '1': 'Corn'})
g_data['Region'] = g_data['Region'].replace({'P': 'Piatt', 'O': 'Occitanie', 'C': 'Crawford', 'M': 'Campo Verde'})

# Filter required columns
data = g_data[['OBJECTID', 'CDL', 'VV', 'VH', 'RVI', 'Ratio', 'NDVI', 'EVI', 'Date', 'Region']]

# Add date-related columns
data['month'] = data.Date.dt.month
data['DOY'] = (data.Date.dt.month * 30) + data.Date.dt.day
data['Year'] = data.Date.dt.year

# Convert dB to linear scale and generate SAR ratios
def db_to_linear(col):
    return 10 ** (0.1 * col)

data[['VH_lin', 'VV_lin']] = data[['VH', 'VV']].apply(db_to_linear)
data['Span'] = data['VH_lin'] + data['VV_lin']
data['Ratio_lin'] = data['VH_lin'] / data['VV_lin']
data['RVI_lin'] = (4 * data['VH_lin']) / (data['VH_lin'] + data['VV_lin'])
data['NRPB_lin'] = (data['VH_lin'] - data['VV_lin']) / (data['VH_lin'] + data['VV_lin'])

# Drop and rename columns
data.drop(columns=['VV', 'VH', 'RVI', 'Ratio'], inplace=True)
data.rename(columns={
    'CDL': 'Crop', 
    'VV_lin': 'VV', 
    'VH_lin': 'VH', 
    'RVI_lin': 'RVI', 
    'Ratio_lin': 'Ratio', 
    'span_lin': 'Span', 
    'NRPB_lin': 'NRPB'
}, inplace=True)

# group data according to region
France_data = data[data['Region'] == 'Occitanie'].reset_index(drop=True)
France_final = France_data.merge(no_DC, on='OBJECTID').drop(columns=['ID_PARCEL'])  # Remove the double-cropping plots data in Occitanie

USA_data = data[data['Region'].isin(['Crawford', 'Piatt'])].reset_index(drop=True)
Brazil_soya = data[(data['Region'] == 'Campo Verde') & (data['Crop'] == 'Soyabean')].reset_index(drop=True)
Brazil_corn = data[(data['Region'] == 'Campo Verde') & (data['Crop'] == 'Corn')].reset_index(drop=True)

# Load and process temperature data
def process_temp_data(filepath):
    temp_data = pd.read_csv(filepath, names=['Date', 'T_max', 'T_min'], skiprows=1)
    temp_data['Date'] = pd.to_datetime(temp_data['Date'])
    return temp_data

USA_temp = process_temp_data('C:/Users/henry/OneDrive/Documents/New_thesis/USA_temp.csv')
France_temp = process_temp_data('C:/Users/henry/OneDrive/Documents/New_thesis/France_temp.csv')
B_soya_temp = process_temp_data('C:/Users/henry/OneDrive/Documents/New_thesis/campo_verde_2015_temp.csv')
B_corn_temp = process_temp_data('C:/Users/henry/OneDrive/Documents/New_thesis/campo_verde_2016_temp.csv')

# Compute Growing Degree Days (GDD)
def compute_GDD(temp_data, t_base=50):
    avg_temp = (temp_data['T_max'] + temp_data['T_min']) / 2
    temp_data['avg'] = np.where(avg_temp > t_base, avg_temp - t_base, 0)
    temp_data['cum_GDD'] = temp_data['avg'].cumsum()
    return temp_data

USA_temp = USA_temp.groupby(USA_temp['Date'].dt.year).apply(compute_GDD).reset_index(drop=True)
France_temp = compute_GDD(France_temp)
B_soya_temp = compute_GDD(B_soya_temp)
B_corn_temp = compute_GDD(B_corn_temp)

# Merge temperature data with the corresponding regions
USA_data = USA_data.merge(USA_temp, on='Date', how='left')
France_final = France_final.merge(France_temp, on='Date', how='left')
Brazil_soya = Brazil_soya.merge(B_soya_temp, on='Date', how='left')
Brazil_corn = Brazil_corn.merge(B_corn_temp, on='Date', how='left')

# Combine data from all regions
reg_data = pd.concat([USA_data, France_final, Brazil_soya, Brazil_corn], ignore_index=True)

# Drop unnecessary columns
reg_data.drop(columns=['T_max', 'T_min', 'avg'], inplace=True)

# Save the final dataframe to a geoparquet file
reg_data.to_parquet('E:/New_thesis/HLS_S1_RUN/HLS_S1.geoparquet')


##### France and Piatt 2019 Crop Cleaning

In [72]:
# Filter data for 2019
France_Piatt_2019 = reg_data[reg_data["Date"].dt.year == 2019].reset_index(drop=True)

# Remove OBJECTIDs with NDVI > 0.5 on the 1st week of June or without NDVI >= 0.5 in August
below_point_5 = France_Piatt_2019[
    (France_Piatt_2019.NDVI <= 0.5) & 
    (France_Piatt_2019.Date.dt.month == 6) & 
    (France_Piatt_2019.Date.dt.day <= 2)
].reset_index(drop=True)

below_point_5_all = France_Piatt_2019[
    France_Piatt_2019['OBJECTID'].isin(below_point_5['OBJECTID'])
].reset_index(drop=True)

# Filter out OBJECTIDs without NDVI >= 0.5 on any acquisition date in August
august_df = below_point_5_all[below_point_5_all['Date'].dt.month == 8]
filtered_df = august_df.groupby('OBJECTID').filter(lambda x: x['NDVI'].max() < 0.5)
potential_crops_2019 = below_point_5_all[
    ~below_point_5_all['OBJECTID'].isin(filtered_df['OBJECTID'])
].reset_index(drop=True)

# Filter for NDVI >= 0.7 on specified dates in June (up to the 2nd)
above_point_7 = France_Piatt_2019[
    (France_Piatt_2019.NDVI >= 0.7) & 
    (France_Piatt_2019.Date.dt.month == 6) & 
    (France_Piatt_2019.Date.dt.day <= 2)
].reset_index(drop=True)

# Remove duplicates farms from concatenated dataframe
non_2_crops = pd.concat([above_point_7, filtered_df], axis=0).drop_duplicates(subset='OBJECTID').reset_index(drop=True)

# Select all other multitemporal data for each farm in the non_2_crops dataframe
all_non_2 = France_Piatt_2019[
    France_Piatt_2019['OBJECTID'].isin(non_2_crops['OBJECTID'])
].reset_index(drop=True)

# Group by Date and Region, calculating the mean of numeric columns
non_2_groups = all_non_2.groupby(['Date', 'Region'], as_index=False).mean(numeric_only=True).set_index('Date')

# Plotting
fig, ax = plt.subplots(figsize=(10, 3))
sns.lineplot(data=non_2_groups, x='DOY', y='NDVI', hue='Region', marker='o', ax=ax)
ax.set_title('NDVI Trends of Potential non-Corn/Soyabean farms by Region in 2019')
ax.set_xlabel('DOY')
ax.set_ylabel('NDVI')

plt.show()

##### 2023 and 2022 Crop Cleaning in Piatt County

In [83]:
# Filter data for 2022 and 2023
Piatt_2022_2023 = reg_data[reg_data["Date"].dt.year >= 2022].reset_index(drop=True)

# Remove OBJECTIDs with NDVI > 0.5 on the 1st week of June
below_point_5 = Piatt_2022_2023[
    (Piatt_2022_2023.NDVI <= 0.5) & 
    (Piatt_2022_2023.Date.dt.month == 6) & 
    (Piatt_2022_2023.Date.dt.day <= 9)
].reset_index(drop=True)

below_point_5_all = Piatt_2022_2023[
    Piatt_2022_2023['OBJECTID'].isin(below_point_5['OBJECTID'])
].reset_index(drop=True)

# Filter out OBJECTIDs without NDVI >= 0.5 on any acquisition date in August and 1st week of September 2023
aug_sep_2023_df = below_point_5_all[
    (below_point_5_all['Date'].dt.month.isin([8, 9])) & 
    (below_point_5_all['Date'].dt.year == 2023)
]

filtered_df = aug_sep_2023_df.groupby('OBJECTID').filter(lambda x: x['NDVI'].max() < 0.5)

potential_crops_2022_2023 = below_point_5_all[
    ~below_point_5_all['OBJECTID'].isin(filtered_df['OBJECTID'])
].reset_index(drop=True)


##### Crop Cleaning for 2020 and 2021 Piatt data

In [87]:
# Filter data for 2020 and 2021
Piatt_2020_2021 = reg_data[reg_data["Date"].dt.year.isin([2020, 2021])].reset_index(drop=True)

# Filter data to only include rows with a Date in August
august_df = Piatt_2020_2021[Piatt_2020_2021['Date'].dt.month == 8]

# Filter out OBJECTIDs without NDVI >= 0.5 on any acquisition date in August
filtered_df = august_df.groupby('OBJECTID').filter(lambda x: x['NDVI'].max() < 0.5)

# Keep only rows that are not in filtered_df
potential_crops_2020_2021 = Piatt_2020_2021[
    ~Piatt_2020_2021['OBJECTID'].isin(filtered_df['OBJECTID'])
].reset_index(drop=True)

In [90]:
# Concatenate all crop-cleaned data from Piatt and France, and previous data from Crawford County and Brazil
reg_data_final = pd.concat([
    potential_crops_2019,
    potential_crops_2020_2021,
    potential_crops_2022_2023,
    reg_data[reg_data['Region'].isin(['Crawford', 'Campo Verde'])]
]).reset_index(drop=True)


In [3]:
# Filter data for the year 2019 and concatenate with specific region data
data_2019 = reg_data_final[reg_data_final["Date"].dt.year == 2019]
data_2019_b_soya = pd.concat([data_2019, reg_data_final[reg_data_final.Region == 'Campo Verde']], ignore_index=True)

# Group by Date, Crop, and Region and calculate the mean
dt_data = data_2019_b_soya.groupby(['Date', 'Crop', 'Region'], as_index=False).mean(numeric_only=True).set_index('Date')

# Group data by Crop type
CDL_group = dt_data.groupby('Crop')

# Select Corn and Soybean data
koo_corn = CDL_group.get_group('Corn')
koo_soybean = CDL_group.get_group('Soyabean')

# Features to plot
x_axis = ['DOY', 'cum_GDD']

# Create a figure with 4 subplots: Corn on the first row, Soybean on the second row
fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(14, 6))

# Loop through Corn and Soybean data to create subplots
for idx, variable in enumerate(x_axis):
    sns.lineplot(data=koo_corn, x=variable, y='NDVI', hue='Region', marker='o', markersize=5, ax=axs[0, idx], legend=False)
    sns.lineplot(data=koo_soybean, x=variable, y='NDVI', hue='Region', marker='o', markersize=5, ax=axs[1, idx], legend=True)

# Set titles and grid for subplots
for i, crop in enumerate(['Corn', 'Soyabean']):
    for j in range(2):
        axs[i, j].set_title(crop)
        axs[i, j].set_xlabel('')
        axs[i, j].set_ylabel('')
        axs[i, j].grid(True)

# Add a single y-axis label for the entire figure
fig.text(0.005, 0.5, 'NDVI', ha='center', va='center', rotation='vertical', fontsize=14)

# Add x-axis labels
fig.text(0.25, 0.01, 'DOY', ha='center', va='center', fontsize=12)
fig.text(0.75, 0.01, 'cum_GDD', ha='center', va='center', fontsize=12)

# Adjust layout to avoid overlap
plt.tight_layout()

# Display the figure
plt.show()


In [ ]:
# Group data by 'Date', 'Crop', and 'Region'
date_group = reg_data_final.groupby(['Date', 'Crop', 'Region'], as_index=False).mean(numeric_only=True).set_index('Date')
date_group['Year'] = date_group['Year'].astype(int)
reg_crop_group = date_group.groupby(['Crop', 'Region'])

# Get Corn and Soybean groups for 'Piatt' region
corn_df = reg_crop_group.get_group(('Corn', 'Piatt'))
soybean_df = reg_crop_group.get_group(('Soyabean', 'Piatt'))

# Group by 'Year' for both Corn and Soybean
corn_masa = corn_df.groupby('Year')
soybean_masa = soybean_df.groupby('Year')

# Plotting variables and confidence intervals
plotting = ['NDVI', 'EVI', 'VH', 'RVI']
conf_interval = [0.05, 0.05, 0.003, 0.025]

# Create a figure with subplots (4 rows, 2 columns)
fig, axs = plt.subplots(nrows=4, ncols=2, figsize=(16, 10))

# Loop through plotting variables for Corn and Soybean
for idx, variable in enumerate(plotting):
    # Corn subplots (left side)
    for name, group in corn_masa:
        upper = group[variable] + conf_interval[idx]
        lower = group[variable] - conf_interval[idx]
        sns.lineplot(data=group, x='DOY', y=variable, label=name, marker='o', markersize=5, ax=axs[idx, 0])
        axs[idx, 0].fill_between(group['DOY'], lower, upper, alpha=0.1)
    axs[idx, 0].set_xlim(180, 310)
    axs[idx, 0].set_title(f'Corn - {variable}')
    axs[idx, 0].grid(True)

    # Soybean subplots (right side)
    for name, group in soybean_masa:
        upper = group[variable] + conf_interval[idx]
        lower = group[variable] - conf_interval[idx]
        sns.lineplot(data=group, x='DOY', y=variable, label=name, marker='o', markersize=5, ax=axs[idx, 1])
        axs[idx, 1].fill_between(group['DOY'], lower, upper, alpha=0.1)
    axs[idx, 1].set_xlim(180, 310)
    axs[idx, 1].set_title(f'Soyabean - {variable}')
    axs[idx, 1].grid(True)

# Collect handles and labels for the legend from the last subplot
handles, labels = axs[-1, 0].get_legend_handles_labels()
handles_soy, labels_soy = axs[-1, 1].get_legend_handles_labels()
handles.extend(handles_soy)
labels.extend(labels_soy)

# Remove legends from individual subplots
for ax in axs.flatten():
    ax.get_legend().remove()

# Add a single legend for the entire figure
fig.legend(handles=handles, labels=labels, loc='lower center', ncol=len(corn_masa), bbox_to_anchor=(0.5, -0.05))

# Add 'DOY' text at the bottom center of the figure
fig.text(0.5, 0.01, 'DOY', ha='center', va='center', fontsize=12)

# Adjust layout to avoid overlap
plt.tight_layout()

# Display the figure
plt.show()


In [128]:
# Function to remove outliers based on the Interquartile Range (IQR)
def ignore_outliers(df, ft):
    q1, q3 = np.percentile(df[ft], [25, 75])
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    return df[(df[ft] > lower_bound) & (df[ft] < upper_bound)]

# Group data by 'Crop'
CDL_group = reg_data_final.groupby('Crop')

# Process each group to remove outliers for specific columns
cdls = []
for name, group in CDL_group:
    for column in ['VV', 'VH']:
        masa = ignore_outliers(group, column)
        cdls.append(masa)

# Combine the cleaned data
clean_data = pd.concat(cdls).reset_index(drop=True)


In [133]:
# Filter clean_data based on NDVI and EVI values
clean_data = clean_data[(clean_data['NDVI'] >= 0.2) & (clean_data['EVI'] >= -1) & (clean_data['EVI'] <= 1)].reset_index(drop=True)

# Replace crop labels with numeric values
clean_data['Crop'] = clean_data['Crop'].replace({'Soyabean': '5', 'Corn': '1'})

# Convert specific columns to category dtype
clean_data[['OBJECTID', 'Crop', 'month']] = clean_data[['OBJECTID', 'Crop', 'month']].astype('category')

# Filter non-soybean data for a specific date
non_soya = clean_data[(clean_data['Crop'] == '5') & (clean_data['Date'] == '2019-06-01 00:00:00')].reset_index(drop=True)

# Create a boolean mask for rows not in non_soya
mask = ~clean_data.set_index(non_soya.columns.tolist()).index.isin(non_soya.set_index(non_soya.columns.tolist()).index)

# Filter clean_data using the mask
ARD = clean_data[mask].reset_index(drop=True)

# Select training and validation subsets
train = ARD[(ARD['Region'] == 'Piatt') & (ARD['Date'].dt.year < 2023)].reset_index(drop=True)
val = ARD[ARD['Date'].dt.year == 2023].reset_index(drop=True)


# Create a dictionary to store all the test subsets
test_dict = {
    'test_1': ARD[ARD['Region'] == 'Crawford'].reset_index(drop=True),
    'test_2': ARD[ARD['Region'] == 'Occitanie'].reset_index(drop=True),
    'test_3': ARD[ARD['Region'] == 'Campo Verde'].reset_index(drop=True),
}

# Find the date when each farm had the highest NDVI value for test_2
max_NDVI_dates = test_dict['test_2'].loc[
    test_dict['test_2'].groupby('OBJECTID', observed=True)['NDVI'].idxmax()
][['OBJECTID', 'Date']]

# Merge the maximum NDVI dates back into the original DataFrame
merged_df = test_dict['test_2'].merge(
    max_NDVI_dates,
    on='OBJECTID',
    suffixes=('', '_Max'),
    how='left'
)

# Helper function to create DataFrames based on the peak NDVI date
def create_dataframes(df, crop_code):
    maturity_df = df[(df['Date'] > df['Date_Max']) & (df['Crop'] == crop_code)].drop(columns='Date_Max').reset_index(drop=True)
    before_peak_df = df[(df['Date'] < df['Date_Max']) & (df['Crop'] == crop_code)].drop(columns='Date_Max').reset_index(drop=True)
    peak_df = df[(df['Date'] == df['Date_Max']) & (df['Crop'] == crop_code)].drop(columns='Date_Max').reset_index(drop=True)
    early_df = pd.concat([before_peak_df, peak_df]).reset_index(drop=True)
    return maturity_df, early_df

# Create DataFrames for Corn and Soybean based on the peak NDVI date
maturity_corn, early_corn = create_dataframes(merged_df, '1')
maturity_soya, early_soya = create_dataframes(merged_df, '5')

# List of NDVI ranges
ndvi_ranges = [round(i / 10, 1) for i in range(2, 9)]  # NDVI ranges from 0.2 to 0.8
ndvi_ranges.append(0.8)  # Last range is >= 0.8

# Helper function to add DataFrames to test_dict based on NDVI ranges
def add_ndvi_dataframes(df, crop_name, phase_name):
    for i, ndvi in enumerate(ndvi_ranges):
        if i < len(ndvi_ranges) - 1:  # For all ranges except the last one
            ndvi_df = df[(df['NDVI'] >= ndvi) & (df['NDVI'] < ndvi + 0.1)].reset_index(drop=True)
        else:  # For the last range, use >= 0.8
            ndvi_df = df[df['NDVI'] >= ndvi].reset_index(drop=True)
        test_dict[f"{crop_name}_{ndvi}_{phase_name}"] = ndvi_df

# Add 'early' and 'maturity' DataFrames for Corn and Soybean to test_dict
add_ndvi_dataframes(early_corn, 'corn', 'early')
add_ndvi_dataframes(early_soya, 'soya', 'early')
add_ndvi_dataframes(maturity_corn, 'corn', 'maturity')
add_ndvi_dataframes(maturity_soya, 'soya', 'maturity')


In [ ]:
# Define features and labels
features = ['Crop', 'VV', 'VH', 'NRPB', 'RVI', 'Ratio', 'Span', 'cum_GDD']
VIs = ['NDVI', 'EVI']

# Select features and labels from the train subset for feature selection
X, Y = train[features], train[VIs]

# Plot a correlation heatmap of independent variables
sns.heatmap(X.corr(), cmap='RdYlGn', linewidths=0.30, annot=True, fmt='.2f')


In [ ]:
# Prepare a figure for the plots
plt.figure(figsize=(12, 6))

for idx, VI in enumerate(VIs):
    # Select features and the current label from the train subset
    X = train[features]
    Y = train[VI]

    # Train the model
    model = lgb.LGBMRegressor(random_state=42, importance_type='gain', verbose=-1, force_col_wise=True)
    model.fit(X, Y)

    # Get feature importances from the model
    feature_importances = model.feature_importances_

    # Sort feature importances in descending order
    indices = np.argsort(feature_importances)[::-1]

    # Create subplots for feature importance
    plt.subplot(1, len(VIs), idx + 1)
    plt.bar(range(len(feature_importances)), feature_importances[indices], color='skyblue')
    plt.xticks(range(len(feature_importances)), X.columns[indices], rotation=90)
    plt.title(f'Feature Importance for {VI}')
    plt.xlim(-1, len(feature_importances))
    plt.ylim(0, 10000)  # Adjust as needed

# Adjust layout and show the plots
plt.tight_layout()
plt.show()

In [ ]:
# sns.set_style('whitegrid')
X_test = {}  # For storing independent variables of the test sets

# Create subplots for side-by-side plotting
fig, axes = plt.subplots(nrows=1, ncols=len(VIs), figsize=(12, 5))

for i, VI in enumerate(VIs):
    # Select features and the current label from the train subset
    X, Y = train[features], train[VI]

    # Train the model
    model = lgb.LGBMRegressor(random_state=42, importance_type='gain', verbose=-1, force_col_wise=True)
    model.fit(X, Y)

    # Calculate permutation importance
    perm_importance = permutation_importance(model, X, Y, random_state=42, scoring='neg_mean_absolute_error')

    # Create a DataFrame for easier analysis
    importance_df = pd.DataFrame({
        'Feature': X.columns,
        'Importance': perm_importance.importances_mean * 10
    })

    # Select features with importance >= 0.10
    sel_features = importance_df[importance_df['Importance'] >= 0.10]['Feature'].tolist()

    # Filter features for training and validation sets
    x_train, x_val = train[sel_features], val[sel_features]

    # Filter features for testing
    for test, df in test_dict.items():
        X_test[test] = df[sel_features]

    # Plot permutation importance
    sorted_idx = np.argsort(importance_df['Importance'])
    axes[i].barh(importance_df['Feature'][sorted_idx], importance_df['Importance'][sorted_idx], color='skyblue')
    axes[i].set_xlabel("Permutation Importance")
    axes[i].set_title(f"Permutation Importance for {VI}")

    # Optional: Print the selected features for each label
    print(f"Selected features for {VI}: {sel_features}")

# Adjust layout and show the plots
plt.tight_layout()
plt.show()


In [ ]:
# Select labels for train, validation, and test sets
y_train, y_val = train[VIs], val[VIs]
Y_test = {test: df[VIs] for test, df in test_dict.items()}

# To store the best hyperparameters and MAE for each VI
best_hps = []
best_MAE = []

# Suppress logging
optuna.logging.set_verbosity(optuna.logging.ERROR)

# Define the objective function for Optuna
def objective(trial, VI):
    # Define hyperparameters
    params = {
        "learning_rate": trial.suggest_float("learning_rate", 0.05, 0.3, step=0.01),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 10, 100, step=10),
        "max_depth": trial.suggest_int("max_depth", 3, 15),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.5, 1, step=0.1),
    }

    # Initialize and fit the model on the training set
    lgb_model = lgb.LGBMRegressor(**params, random_state=42)
    lgb_model.fit(x_train, y_train[VI])

    # Predict on the validation set
    y_pred_val = lgb_model.predict(x_val)

    # Calculate Mean Absolute Error (MAE)
    MAE = mean_absolute_error(y_val[VI], y_pred_val)
    
    return MAE

# Optimize hyperparameters for each VI
for VI in VIs:
    study = optuna.create_study(direction='minimize', sampler=optuna.samplers.RandomSampler(seed=42))
    study.optimize(lambda trial: objective(trial, VI), n_trials=100)

    best_hps.append(study.best_params)
    best_MAE.append(study.best_value)


In [ ]:
# Initialize lists to store predictions, models, and residuals
VI_preds, models, residuals = [], [], []
results = []

# Function to calculate root mean squared error
def root_mean_squared_error(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

# Flag to indicate if columns should be dropped
drop_columns = False

# Loop through each test set and corresponding labels
for i, (test_name, (x, y)) in enumerate(zip(X_test.keys(), zip(X_test.values(), Y_test.values()))):
    # Check if the iteration is after the third key-value pair
    if i >= 3:
        drop_columns = True
    
    for VI, best_params in zip(VIs, best_hps):
        # Adjust features if the drop_columns flag is set
        if drop_columns:
            x_adj = x.drop(x.columns[0], axis=1)
            x_train_adj = x_train.drop(x_train.columns[0], axis=1)
        else:
            x_adj = x
            x_train_adj = x_train

        # Fit the model
        model = lgb.LGBMRegressor(random_state=42, importance_type='gain', **best_params)
        model.fit(x_train_adj, y_train[VI])
        
        # Predict on the test set
        y_pred = model.predict(x_adj)
        y_pred_df = pd.DataFrame(y_pred, columns=[f'{VI}_pred'])
        
        # Calculate residuals
        residual = y[VI] - y_pred
        residual_df = pd.DataFrame(residual, columns=[f'{VI}_res'])
        
        # Store predictions and residuals
        VI_preds.append(y_pred_df)
        residuals.append(residual_df)
        
        # Calculate evaluation metrics
        mae = mean_absolute_error(y[VI], y_pred)
        rmse = root_mean_squared_error(y[VI], y_pred)
        r2 = model.score(x_adj, y[VI])
        bias = np.mean(residual)

        # Store the result for this label and test set
        results.append({
            'test_name': test_name,
            'VI': VI,
            'MAE': round(mae, 3),
            'RMSE': round(rmse, 3),
            'bias': round(bias, 4),
            'R2': round(r2, 3),
        })

# Convert results to a DataFrame
results_df = pd.DataFrame(results)
results_df['test_name'] = results_df['test_name'].replace({'test_1': 'Crawford', 'test_2': 'Occitanie', 'test_3': 'Campo Verde'})


In [ ]:
region_eva= results_df.head(6) # evaluation metrics for geographical area analysis

In [231]:
# Extract test results containing '0.' and derive range and stage columns
stages_results = results_df[results_df['test_name'].str.contains('0.')]
stages_results['range'] = stages_results['test_name'].str[5:8].replace({'0.8': '>=0.8'})
stages_results['stage'] = stages_results['test_name'].str[9:]

# Filter for corn and soya tests with NDVI
corn_NDVI = stages_results[(stages_results['test_name'].str.contains('corn')) & (stages_results['VI'] == 'NDVI')]
soya_NDVI = stages_results[(stages_results['test_name'].str.contains('soya')) & (stages_results['VI'] == 'NDVI')]

# Define color palette
colors = ['#2E8B57', '#ff7f0e']  # green and orange

# Group the dataframe by stage 
grouped_df = corn_NDVI.groupby('stage')

# Create subplots for two bar graphs side by side with shared y-axis
fig, axs = plt.subplots(1, 2, figsize=(10, 4), sharey=True)

# Loop through each unique stage value and plot
for idx, (stage, data) in enumerate(grouped_df):
    axs[idx].bar(data['range'], data['MAE'], color=colors[idx])
    if idx == 1:
        axs[idx].invert_xaxis()  # Reverse the x-axis for the second plot
        axs[idx].tick_params(left=False, labelleft=False)  # Remove y-axis ticks for the second plot
    axs[idx].grid(True, axis='y', linestyle='--', linewidth=0.5)  # Add horizontal grid lines only

# Set common labels and y-axis range
fig.text(0.53, 0.005, 'Range', ha='center')
fig.text(0.001, 0.5, 'MAE', va='center', rotation='vertical')
for ax in axs:
    ax.set_ylim(0.00, 0.20)

# Adjust layout to prevent overlap and show plot
plt.tight_layout()
plt.show()

In [331]:
# Filter test_2 for OBJECTIDs that appear 10 times
ten_dates = test_dict['test_2'][test_dict['test_2']['OBJECTID'].map(test_dict['test_2']['OBJECTID'].value_counts()) == 10]

# Filter max_NDVI_dates for specific date and match OBJECTIDs in both datasets
aug_17 = max_NDVI_dates[max_NDVI_dates.Date == '2019-08-17 00:00:00']
aug_ten = ten_dates[ten_dates.OBJECTID.isin(aug_17.OBJECTID)].reset_index(drop=True)

# Concatenate predictions and residuals along columns
pred_res = pd.concat([pd.concat(VI_preds, axis=1), pd.concat(residuals, axis=1)], axis=1)

# Combine data for visualization
vis_data = pd.concat([test_dict['test_2'][['OBJECTID', 'Date']], pred_res], axis=1)
aug_ten = aug_ten.merge(vis_data, on=['OBJECTID', 'Date'])

# Drop unwanted columns
unwanted = ['VV', 'VH', 'NRPB', 'RVI', 'Ratio', 'Span']
aug_ten.drop(unwanted, axis=1, inplace=True)

# Select and group corn data
corn_sel = aug_ten[aug_ten['Crop'] == '1'].sample(n=4, random_state=42)
corn_sel_all = aug_ten[aug_ten.OBJECTID.isin(corn_sel.OBJECTID)].reset_index(drop=True)
corn_group = corn_sel_all.groupby('OBJECTID', observed=True)

# Select and group soya data
soya_sel = aug_ten[aug_ten['Crop'] == '5'].sample(n=4, random_state=10)
soya_sel_all = aug_ten[aug_ten.OBJECTID.isin(soya_sel.OBJECTID)].reset_index(drop=True)
soya_group = soya_sel_all.groupby('OBJECTID', observed=True)

# Create a figure and axes with 2 rows and 2 columns (4 subplots)
fig, axes = plt.subplots(2, 2, figsize=(12, 6))

# Flatten the axes array for easier iteration
axes = axes.flatten()

# Iterate over the grouped data for each OBJECTID
for idx, (objectid, group) in enumerate(soya_group):
    # Plot the time series for the actual NDVI and predicted NDVI for each OBJECTID
    axes[idx].plot(group['Date'], group['NDVI'], label='Actual NDVI', color='blue', marker='o')
    axes[idx].plot(group['Date'], group['NDVI_pred'], label='Predicted NDVI', color='red', marker='x')
    # Rotate date labels for readability
    axes[idx].tick_params(axis='x', rotation=45)

# Add a single legend for all subplots
fig.legend(['Actual NDVI', 'Predicted NDVI'], loc='upper center', ncol=2)

# Add shared axis labels for the entire figure
fig.supxlabel('Date', fontsize=14)
fig.supylabel('NDVI', fontsize=14)

# Adjust layout to avoid overlapping
plt.tight_layout(rect=[0, 0, 1, 0.96])  # Leave space at the top for the legend

# Display the plot
plt.show()

In [261]:
# Concatenate predictions and residuals along columns
pred_res = pd.concat([pd.concat(VI_preds, axis=1), pd.concat(residuals, axis=1)], axis=1)

# Combine data for visualization
vis_data = pd.concat([test_dict['test_1'][['OBJECTID', 'Date']], pred_res], axis=1)
g_data_1 = g_data.merge(vis_data, on=['OBJECTID', 'Date'])

# Group the data by date
vis_groups = g_data_1.groupby('Date')

# Define colormap
cmap = 'tab20_r'

def plot_ndvi_groups(vis_groups, date: str, cmap: str) -> None:
    """
    Plots NDVI values for specified date across multiple columns.

    Args:
    vis_groups (DataFrameGroupBy): Grouped data containing NDVI values.
    date (str): The date for which to plot the NDVI values.
    cmap (str): Colormap to use for the plots.
    
    Returns:
    None: Displays the plots.
    """
    NDVI_columns = vis_groups.get_group(date).filter(like='NDVI')
    fig, axs = plt.subplots(1, len(NDVI_columns.columns), figsize=(15, 6))

    for i, column in enumerate(NDVI_columns.columns):
        vis_groups.get_group(date).plot(
            y=column,
            ax=axs[i],
            legend=False,
            cmap=cmap,
            vmax=1.0,
            vmin=0.1
        ).set_title(f'{column}, {date[:10]}')
    
    # Add colorbar to the figure
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=0.0, vmax=1.0))
    fig.colorbar(sm, ax=axs, fraction=0.04, pad=0.1, location='bottom').set_label('NDVI')

    # Display the plot
    plt.tight_layout()
    plt.show()

# Plot NDVI groups for each date
for name, group in vis_groups:
    plot_ndvi_groups(vis_groups, name, cmap)
